In [1]:
import os
import pandas as pd
import shap
import xgboost
import numpy as np
import csv
import sklearn



from xgboost import plot_tree
import matplotlib.pyplot as plt

# Function to calculate the accuracy of the model
def calculate_acc(pred, labels):
    matches = 0
    for i in range(len(labels)):
      if pred[i] == labels[i]:
        matches += 1    
    return matches/len(labels)


def get_all_shap(shap_values, class_names, max_display=None, silent=None):
    
    """Get the values from the SHAP summary plot, only for multiclass

    Parameters
    ----------
    shap_values : numpy.array
        For single output explanations this is a matrix of SHAP values (# samples x # features).
        For multi-output explanations this is a list of such matrices of SHAP values.

    max_display : int
        How many top features to include in the plot (default is 20, or 7 for interaction plots)

    silent : bool
        Print out the calculations
    """
    
    # Start values
    if max_display is None:
        max_display = 2

    if silent is None:
        silent = True



    # get the id of the most important features  [not important: most important]
    feature_order = np.argsort(np.sum(np.mean(np.abs(shap_values), axis=0), axis=0))

    # reverse the array tp [most important, ..., less important]
    feature_order = np.flip(feature_order)

    # get a list with the most important feature names
    feature_names = ['label']
    for i in feature_order[0:max_display]:
        feature_names.append(df.columns[i+1])
        
    # print out the most important features
    if not silent:
        print("Most important features:")
        print(feature_names[1:])

    # get the names of the class
    #class_names = model.classes_

    # get only the ids to num_max features
    feature_inds = feature_order[:max_display]

    # get a list with id's of the classes
    class_inds = np.arange(len(class_names))

    # create empty list to store all shap values
    all_shap = []

    # append the feature_names to the list
    all_shap.append(feature_names)


    # i = counter, ind = class_inds 
    for i,ind in enumerate(class_inds):

        # get an array with all the shap-values [shap_value_feature1, shap_value_feature2...]
        global_shap_values = np.abs(shap_values[ind]).mean(0)
        values = global_shap_values[feature_inds]

        # get the class for the array with the shap values
        label=class_names[ind]

        # add the class to the array
        values = np.concatenate((np.array([label]),values) , axis=0)

        # store the values
        all_shap.append(values.tolist())
        
    return all_shap

In [2]:
path_to_data = "/Users/tim/Documents/HTW_Berlin/2_WS_21_22/projektarbeit_XAI/data_02/Frequency Features/frequencyFeatures.csv"

#read in the data using pandas
df = pd.read_csv(path_to_data)

# Delete the two features
df = df.drop(['Label', 'Timestamp'], axis=1)

# Delete label off
df = df.iloc[250:, :]
df



,ID,xAcc010Hz,xAcc015Hz,xAcc020Hz,xAcc025Hz,xAcc030Hz,xAcc035Hz,xAcc040Hz,xAcc045Hz,xAcc050Hz,...,snd2275Hz,snd2300Hz,snd2325Hz,snd2350Hz,snd2375Hz,snd2400Hz,snd2425Hz,snd2450Hz,snd2475Hz,snd2500Hz
250,2,454.729201,373.662293,841.151061,740.903054,335.169305,1277.359858,2338.590535,9415.907494,11120.59792,...,2.081464,2.573535,2.920927,4.982745,3.420132,0.867515,4.606851,2.301164,3.850050,2.092610
251,2,183.645769,320.688104,604.463776,746.636921,417.433785,1024.675231,2260.370181,9701.824416,12373.64882,...,1.899494,4.541002,5.725692,3.447297,2.802118,1.385863,2.985951,3.087122,5.687176,1.633761
252,2,192.824680,197.466285,237.154490,545.014362,524.603158,858.507832,356.197129,8766.841994,12655.39475,...,3.947479,4.772883,9.668906,3.489168,2.272290,2.577190,2.675607,5.028594,5.148495,2.285906
253,2,444.692309,607.832208,311.244219,598.615257,557.446143,644.497740,1136.291001,9139.170561,12116.00045,...,3.838190,3.088682,11.496789,2.656639,2.551696,2.951968,3.115344,3.571959,2.862464,4.497884
254,2,806.189855,213.300371,719.645236,379.386502,261.768009,13.231422,569.551468,9577.904815,12135.93629,...,2.575419,2.697834,7.483828,1.691467,2.032943,1.919057,1.855661,0.584894,0.871019,4.263095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,8,734.684640,385.405156,2422.030460,3666.688023,1331.591958,295.834974,615.946313,8784.043068,12981.34632,...,1.810877,4.199799,3.196192,4.824971,5.501506,2.569931,2.769503,2.864246,4.875627,3.097777
1996,8,427.768727,478.715371,1667.903389,3555.054589,1412.465945,546.736119,755.346273,9108.426555,13018.32149,...,5.296502,1.327645,2.921181,4.135513,7.574796,4.394050,1.986867,3.715002,7.698977,3.895627
1997,8,183.683706,244.413614,1889.702322,3486.772611,1175.411282,844.245959,1166.437422,9453.092350,13172.74271,...,8.563501,3.625357,9.565385,2.401314,8.655602,3.706591,4.951512,3.277185,9.806276,3.751800
1998,8,231.417381,290.299651,2107.940465,3431.216806,1618.341691,977.230736,577.816480,9787.239494,13983.28469,...,8.862310,4.188897,13.152540,1.534665,6.803201,1.311985,3.673036,2.172725,7.631238,0.765262


In [3]:
x = df.iloc[:,1:]
y = df.iloc[:,0]

In [4]:
from sklearn.model_selection import train_test_split

# Split the data in 80% Train and 20% Test Sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

# data must be a numpy array
np_y_test = y_test.to_numpy()

In [5]:
# Train the KNN model
from sklearn import neighbors
n_neighbors = 15

knn = neighbors.KNeighborsClassifier(n_neighbors,weights='distance')
knn.fit(x_train,y_train)

# Produce the SHAP values
knn_explainer = shap.KernelExplainer(knn.predict,x_test)
knn_shap_values = knn_explainer.shap_values(x_test, nsamples= 10)

Using 350 background data samples could cause slower run times. Consider using shap.kmeans(data, K) to summarize the background as K weighted samples.


  0%|          | 0/350 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 5 iterations, i.e. alpha=2.501e-03, with an active set of 5 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 6 iterations, i.e. alpha=2.175e-03, with an active set of 6 regressors, and the smallest cholesky pivot element being 1.054e-08. Red

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 1 iterations, i.e. alpha=8.535e-02, with an active set of 1 regressors, and the smallest cholesky pivot element being 2.107e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=7.504e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Red

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.200e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.200e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.054e-08. Red

...

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=8.612e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=4.306e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Red

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=3.662e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=3.112e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Red

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 1 iterations, i.e. alpha=2.920e-01, with an active set of 1 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps par

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=8.078e-03, with an active set of 2 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=4.039e-03, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.490e-08. Red

/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 1 iterations, i.e. alpha=3.184e-02, with an active set of 1 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=1.592e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 1.490e-08. Red